<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading

### ⚠️ Eikon requires a paid subscription, I haven't get it so the following wasn't tested

**Chapter 03 &mdash; Working with Financial Data**

### Working with Open Data Sources - Refinitiv Eikon Data API

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('../env.cfg')

`pip install eikon` &mdash; Requires a paid subscription.

In [ ]:
import eikon as ek

In [ ]:
ek.set_app_key(config['eikon']['app_key'])

In [ ]:
help(ek)

### Retrieving Historical Structured Data

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
symbols = ['AAPL.O', 'MSFT.O', 'GOOG.O']

In [ ]:
data = ek.get_timeseries(symbols,
                         start_date='2020-01-01',
                         end_date='2020-05-01',
                         interval='daily',
                         fields=['*'])

In [ ]:
data.keys()

In [ ]:
type(data['AAPL.O'])

In [ ]:
data['AAPL.O'].info()

In [ ]:
data['AAPL.O'].tail()

In [ ]:
%%time
data = ek.get_timeseries(symbols,
                         start_date='2020-05-05',
                         end_date='2020-05-06',
                         interval='minute',
                         fields='*')

In [ ]:
print(data['GOOG.O'].loc['2020-05-05 16:00:00':
                         '2020-05-05 16:04:00'].round(1))

In [ ]:
for sym in symbols:
    print('\n' + sym + '\n', data[sym].iloc[-300:-295].round(1))

In [ ]:
%%time
data = ek.get_timeseries(symbols[0],
                         start_date='2020-05-05 15:00:00',
                         end_date='2020-05-05 16:00:00',
                         interval='tick',
                         fields=['*'])

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
resampled = data.resample('30s', label='right').agg(
            {'VALUE': 'last', 'VOLUME': 'sum'}) # <6>

In [ ]:
resampled.tail()

### Retrieving Historical Unstructured Data

In [ ]:
headlines = ek.get_news_headlines(query='R:AAPL.O macbook',
                                  count=5,
                                  date_from='2020-4-1',
                                  date_to='2020-5-1')

In [ ]:
headlines

In [ ]:
story = headlines.iloc[0]

In [ ]:
story

In [ ]:
news_text = ek.get_news_story(story['storyId'])

In [ ]:
from IPython.display import HTML

In [ ]:
HTML(news_text)

## Storing Financial Data Efficiently

### Storing DataFrame Objects

In [ ]:
from sample_data import generate_sample_data

In [ ]:
print(generate_sample_data(rows=5, cols=4))

In [ ]:
%time data = generate_sample_data(rows=5e6, cols=10).round(4)

In [ ]:
data.info()

In [ ]:
h5 = pd.HDFStore('data/data.h5', 'w')

In [ ]:
%time h5['data'] = data

In [ ]:
h5

In [ ]:
h5.close()

In [ ]:
h5 = pd.HDFStore('data/data.h5', 'r')

In [ ]:
%time data_copy = h5['data']

In [ ]:
data_copy.info()

In [ ]:
h5.close()

In [ ]:
rm data/data.h5

In [ ]:
%time data.to_hdf('data/data.h5', 'data', format='table')

In [ ]:
ls -n data/data.*

In [ ]:
%time data_copy = pd.read_hdf('data/data.h5', 'data')

In [ ]:
data_copy.info()

In [ ]:
import tables as tb

In [ ]:
h5 = tb.open_file('data/data.h5', 'r')

In [ ]:
h5

In [ ]:
h5.root.data.table[:3]

In [ ]:
h5.close()

In [ ]:
rm data/data.h5

### Using TsTables

In [ ]:
%%time
data = generate_sample_data(rows=2.5e6, cols=5,
                            freq='1s').round(4)

In [ ]:
data.info()

In [ ]:
import tstables

In [ ]:
import tables as tb

In [ ]:
class desc(tb.IsDescription):
    ''' Description of TsTables table structure.
    '''
    timestamp = tb.Int64Col(pos=0)
    No0 = tb.Float64Col(pos=1)
    No1 = tb.Float64Col(pos=2)
    No2 = tb.Float64Col(pos=3)
    No3 = tb.Float64Col(pos=4)
    No4 = tb.Float64Col(pos=5)

In [ ]:
h5 = tb.open_file('data/data.h5ts', 'w')

In [ ]:
ts = h5.create_ts('/', 'data', desc)

In [ ]:
h5

In [ ]:
%time ts.append(data)

In [ ]:
# h5

In [ ]:
import datetime

In [ ]:
start = datetime.datetime(2021, 1, 2)

In [ ]:
end = datetime.datetime(2021, 1, 3)

In [ ]:
%time subset = ts.read_range(start, end)

In [ ]:
start = datetime.datetime(2021, 1, 2, 12, 30, 0)

In [ ]:
end = datetime.datetime(2021, 1, 5, 17, 15, 30)

In [ ]:
%time subset = ts.read_range(start, end)

In [ ]:
subset.info()

In [ ]:
h5.close()

In [ ]:
rm data/*

### Storing Data with SQLite3

In [ ]:
%time data = generate_sample_data(1e6, 5, '1min').round(4)

In [ ]:
data.info()

In [ ]:
import sqlite3 as sq3

In [ ]:
con = sq3.connect('data/data.sql')

In [ ]:
%time data.to_sql('data', con)

In [ ]:
ls -n data/data.*

In [ ]:
query = 'SELECT * FROM data WHERE No1 > 105 and No2 < 108'

In [ ]:
%time res = con.execute(query).fetchall()

In [ ]:
res[:5]

In [ ]:
len(res)

In [ ]:
con.close()

In [ ]:
rm data/*

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>